In [14]:
import os
import numpy as np
import git
import mlflow
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [15]:
# --- 1. Метрики и Scoring Functions ---

def calculate_metrics(y_true, y_pred):
    """
    Вычисляет расширенный набор метрик для RUL.
    y_true, y_pred: torch.Tensor или numpy array
    """
    if isinstance(y_true, torch.Tensor):
        y_true = y_true.detach().cpu().numpy()
    if isinstance(y_pred, torch.Tensor):
        y_pred = y_pred.detach().cpu().numpy()
        
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    # Разница
    d = y_pred - y_true
    
    # 1. MAE
    mae = np.mean(np.abs(d))
    
    # 2. RMSE
    rmse = np.sqrt(np.mean(d**2))
    
    # 3. MAPE
    mape = np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1.0))) * 100

    # 4. PHM08 Score (NASA Scoring Function) [web:PHM08_Challenge]
    # Функция асимметрична: ранние предсказания (d < 0) штрафуются меньше, чем поздние (d > 0)
    # Формула: sum(exp(-d/13) - 1 если d < 0, иначе exp(d/10) - 1)
    # *Внимание: в условии было указано score = sum(...) / n. Обычно в PHM08 используют просто sum,
    # но для сопоставимости метрик лучше использовать среднее (mean) или следовать условию задачи.
    # Здесь реализуем согласно вашему ТЗ: делим на n.
    n = len(d)
    scores = np.where(d < 0, np.exp(-d/13) - 1, np.exp(d/10) - 1)
    phm08_score = np.sum(scores) / n
    
    return {"mae": mae, "rmse": rmse, "mape": mape, "phm08_score": phm08_score}

In [16]:
# --- 2. Адаптивная модель (Transfer Learning Ready) ---

class AdaptiveLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim_1=64, hidden_dim_2=32, output_dim=1, dropout_prob=0.2):
        super(AdaptiveLSTMModel, self).__init__()
        
        # === АДАПТИВНЫЙ ВХОДНОЙ СЛОЙ (ADAPTER) ===
        # Проецируем входные признаки (любого кол-ва) в фиксированное скрытое пространство (hidden_dim_1).
        # При Transfer Learning мы заменим только этот слой.
        self.input_adapter = nn.Linear(input_dim, hidden_dim_1)
        
        # === BACKBONE (ЯДРО МОДЕЛИ) ===
        # LSTM слои теперь принимают hidden_dim_1, а не input_dim.
        # Это позволяет весам LSTM оставаться валидными даже если input_dim изменится.
        self.backbone_lstm1 = nn.LSTM(hidden_dim_1, hidden_dim_1, batch_first=True)
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.backbone_lstm2 = nn.LSTM(hidden_dim_1, hidden_dim_2, batch_first=True)
        self.dropout2 = nn.Dropout(dropout_prob)
        
        # Финальная "голова"
        self.fc = nn.Linear(hidden_dim_2, output_dim)

    def forward(self, x):
        # x: (batch_size, seq_len, input_dim)
        
        # 1. Адаптация входа
        x_embedded = self.input_adapter(x) # -> (batch_size, seq_len, hidden_dim_1)
        
        # 2. Проход через Backbone
        lstm1_out, _ = self.backbone_lstm1(x_embedded)
        out = self.dropout1(lstm1_out)
        
        lstm2_out, _ = self.backbone_lstm2(out)
        
        # Берем выход последнего временного шага
        last_hidden_state = lstm2_out[:, -1, :] 
        out = self.dropout2(last_hidden_state)
        
        # 3. Финальный прогноз
        final_output = self.fc(out)
        return final_output

In [17]:
# Указываем MLflow, куда отправлять данные
mlflow.set_tracking_uri("http://213.21.252.250:5000")

# Задаем имя эксперимента
mlflow.set_experiment("LSTM_TransferLearning_Ready")

# --- Получаем хеш коммита Git ---
try:
    repo = git.Repo(search_parent_directories=True)
    git_commit_hash = repo.head.object.hexsha
except Exception as e:
    git_commit_hash = "N/A" # На случай, если скрипт запущен не из Git-репозитория
    print(f"Warning: Could not get git commit hash. {e}")

print(f"Current Git Commit Hash: {git_commit_hash}")

# --- Параметры, которые нужно логировать ---
# Параметры из скрипта нарезки данных (sample_creator)
data_params = {
    "window_size": 50,
    "step": 1,
    "sampling_rate": 10
}

# Гиперпараметры модели
model_params = {
    "epochs": 10,
    "batch_size": 128,
    "validation_split": 0.2,
    "optimizer": "adam",
    "loss": "mean_squared_error",
    "lr": 0.002,
    "hidden_dim_1": 32,
    "hidden_dim_2": 16,
    "dropout": 0.2
}

Current Git Commit Hash: 068edc6a317958c4f3595aafd2a7c2df5c989afc


In [18]:
def load_and_merge_data(npz_units):
      sample_array_lst = []
      label_array_lst = []
      for npz_unit in npz_units:
        loaded = np.load(npz_unit)
        sample_array_lst.append(loaded['sample'])
        label_array_lst.append(loaded['label'])
      sample_array = np.dstack(sample_array_lst)
      label_array = np.concatenate(label_array_lst)
      sample_array = sample_array.transpose(2, 0, 1)
      return sample_array, label_array

processed_dir = '../data/processed/'

# Собираем пути к файлам для train и test
train_files = [os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.startswith(('Unit2_', 'Unit5_', 'Unit10_', 'Unit16_', 'Unit18_', 'Unit20_'))]
test_files = [os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.startswith(('Unit11_', 'Unit14_', 'Unit15_'))]
print(train_files)

# Загружаем данные
X_train, y_train = load_and_merge_data(train_files)
X_test, y_test = load_and_merge_data(test_files)

# Определяем форму входных данных из X_train
n_timesteps, n_features = X_train.shape[1], X_train.shape[2]

print('Размер обучающей выборки (X):', X_train.shape)
print('Размер обучающей выборки (y):', y_train.shape)
print('Размер тестовой выборки (X):', X_test.shape)
print('Размер тестовой выборки (y):', y_test.shape)

['../data/processed/Unit16_win50_str1_smp10.npz', '../data/processed/Unit5_win50_str1_smp10.npz', '../data/processed/Unit18_win50_str1_smp10.npz', '../data/processed/Unit20_win50_str1_smp10.npz', '../data/processed/Unit2_win50_str1_smp10.npz', '../data/processed/Unit10_win50_str1_smp10.npz']


KeyboardInterrupt: 

In [ ]:
with mlflow.start_run():
    print("Starting MLflow run...")

    # --- Логируем параметры ---
    mlflow.log_params(data_params)
    mlflow.log_params(model_params)
    mlflow.set_tag("git_commit", git_commit_hash)
    print("Parameters logged.")

    
    # --- Определяем модель LSTM на PyTorch ---
    class LSTMModel(nn.Module):
        def __init__(self, input_dim, hidden_dim_1, hidden_dim_2, output_dim=1, dropout_prob=0.2):
            super(LSTMModel, self).__init__()
            # Первый LSTM слой
            self.lstm1 = nn.LSTM(input_dim, hidden_dim_1, batch_first=True)
            # batch_first=True очень важен, чтобы входные данные имели формат (batch, seq, feature), как в Keras
            
            self.dropout1 = nn.Dropout(dropout_prob)
            
            # Второй LSTM слой
            # Он принимает на вход скрытое состояние первого слоя (hidden_dim_1)
            self.lstm2 = nn.LSTM(hidden_dim_1, hidden_dim_2, batch_first=True)
            
            self.dropout2 = nn.Dropout(dropout_prob)
            
            # Полносвязный слой для финального прогноза
            self.fc = nn.Linear(hidden_dim_2, output_dim)

        def forward(self, x):
            # Первый LSTM слой
            # LSTM возвращает output и кортеж (hidden_state, cell_state)
            # Нам нужен output для следующего слоя
            lstm1_out, _ = self.lstm1(x)
            
            # Dropout
            out = self.dropout1(lstm1_out)
            
            # Второй LSTM слой
            # Нам нужен только выход последнего временного шага
            lstm2_out, _ = self.lstm2(out)
            last_hidden_state = lstm2_out[:, -1, :] # Берем выход последнего элемента последовательности
            
            # Dropout
            out = self.dropout2(last_hidden_state)
            
            # Полносвязный слой
            final_output = self.fc(out)
            return final_output

    # --- Подготовка данных для PyTorch ---
    # 1. Преобразуем numpy массивы в torch тензоры
    X_train_tensor = torch.from_numpy(X_train).float()
    y_train_tensor = torch.from_numpy(y_train).float().view(-1, 1) # Убедимся, что форма (batch_size, 1)
    X_test_tensor = torch.from_numpy(X_test).float()
    y_test_tensor = torch.from_numpy(y_test).float().view(-1, 1)

    # 2. Создаем датасеты
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    # 3. Разделяем на обучающую и валидационную выборки вручную
    val_split = model_params['validation_split']
    dataset_size = len(train_dataset)
    val_size = int(val_split * dataset_size)
    train_size = dataset_size - val_size
    train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

    # 4. Создаем загрузчики данных (DataLoader), которые будут подавать данные батчами
    train_loader = DataLoader(dataset=train_subset, batch_size=model_params['batch_size'], shuffle=True)
    val_loader = DataLoader(dataset=val_subset, batch_size=model_params['batch_size'])
    test_loader = DataLoader(dataset=test_dataset, batch_size=model_params['batch_size'])

    # Инициализация модели
    device = torch.device("cpu") # Требование: работать на CPU
    
    model = AdaptiveLSTMModel(
        input_dim=n_features,
        hidden_dim_1=model_params["hidden_dim_1"],
        hidden_dim_2=model_params["hidden_dim_2"],
        dropout_prob=model_params["dropout"]
    ).to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=model_params["lr"])

    # Цикл обучения
    for epoch in range(model_params["epochs"]):
        model.train()
        train_losses = []
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        avg_train_loss = np.mean(train_losses)
        mlflow.log_metric("train_mse_loss", avg_train_loss, step=epoch)

        # Валидация
        model.eval()
        val_preds = []
        val_targets = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                val_preds.append(outputs.numpy())
                val_targets.append(labels.numpy())
        
        val_preds = np.concatenate(val_preds)
        val_targets = np.concatenate(val_targets)
        
        # Расчет всех метрик
        val_metrics = calculate_metrics(val_targets, val_preds)
        
        # Логирование метрик валидации
        for name, value in val_metrics.items():
            mlflow.log_metric(f"val_{name}", value, step=epoch)
            
        print(f"Epoch {epoch+1}/{model_params['epochs']} | Train Loss: {avg_train_loss:.2f} | Val MAE: {val_metrics['mae']:.2f} | PHM08: {val_metrics['phm08_score']:.2f}")

    # --- 5. Финальный тест и сохранение ---
    print("\nEvaluating on Test Set...")
    model.eval()
    test_preds = []
    test_targets = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            test_preds.append(outputs.numpy())
            test_targets.append(labels.numpy())
    
    test_preds = np.concatenate(test_preds)
    test_targets = np.concatenate(test_targets)
    
    test_metrics = calculate_metrics(test_targets, test_preds)
    print(f"Test Metrics: {test_metrics}")
    
    # Логируем финальные метрики с префиксом test_
    for name, value in test_metrics.items():
        mlflow.log_metric(f"test_{name}", value)

    # 1. Сохранение ПОЛНОЙ модели
    mlflow.pytorch.log_model(model, "full_model")
    
    # 2. Сохранение BACKBONE (State Dict без входного слоя) для Transfer Learning
    # Исключаем веса input_adapter, чтобы клиент мог инициализировать свои
    backbone_state_dict = {k: v for k, v in model.state_dict().items() if "input_adapter" not in k}
    torch.save(backbone_state_dict, "backbone.pth")
    mlflow.log_artifact("backbone.pth", artifact_path="transfer_learning_artifacts")
    
    print("Run Complete. Artifacts logged.")

Starting MLflow run...
Parameters logged.
Epoch 1/7 | Train Loss: 666.80 | Val MAE: 19.15 | PHM08: 8.05
Epoch 2/7 | Train Loss: 518.58 | Val MAE: 19.15 | PHM08: 8.04
Epoch 3/7 | Train Loss: 517.20 | Val MAE: 19.15 | PHM08: 8.05
Epoch 4/7 | Train Loss: 414.75 | Val MAE: 7.32 | PHM08: 1.49
Epoch 5/7 | Train Loss: 80.39 | Val MAE: 5.37 | PHM08: 0.84
Epoch 6/7 | Train Loss: 72.07 | Val MAE: 4.74 | PHM08: 0.69
Epoch 7/7 | Train Loss: 68.47 | Val MAE: 4.63 | PHM08: 0.62

Evaluating on Test Set...
Test Metrics: {'mae': 4.0629144, 'rmse': 5.3320966, 'mape': 36561420800.0, 'phm08_score': 0.5586638414639016}


2025/12/20 22:13:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/20 22:14:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run Complete. Artifacts logged.
🏃 View run thoughtful-wasp-743 at: http://213.21.252.250:5000/#/experiments/2/runs/3d9fc09d7c7d436ca54132cbd05102c6
🧪 View experiment at: http://213.21.252.250:5000/#/experiments/2
